# Algorithm Implement

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
time_period = 15
class Q_Network(nn.Module):
    '''
    The input of this network should have shape (num_frame, 80, 80)
    '''

    def __init__(self, num_frame, num_action, N):
        super(Q_Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=num_frame, out_channels=32, kernel_size=(2,1), stride=1, padding=2)  # 16, 20, 20
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(2,1), stride=1)  # 32, 9, 9
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(2,1), stride=1)  # 32, 9, 9
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(2,1), stride=1)  # 32, 9, 9
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(2,2), stride=1)  # 32, 9, 9
        self.pool = nn.AvgPool2d(kernel_size=(2,1))
        self.fc1 = nn.Linear(576, 256)
        self.fc2 = nn.Linear(256, num_action*N)
        self.action_size = num_action
        self.N = N

    def forward(self, image):
        x = F.relu(self.pool(self.conv1(image)))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.relu(self.pool(self.conv3(x)))
        x = x.view(-1, 576)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.view(-1, self.action_size, self.N)
        return x

In [133]:
from torchsummary import summary

# Data Loading

In [ ]:
pwd

In [2]:
import pandas as pd
import pickle
file = open('../../FinBert/stock_data_full.bin', 'rb')
data = pickle.load(file)
file.close()

In [3]:
codes = ['AAPL','AMZN','C','GOOG','JPM','NFLX','PLTR']
for i in range(len(codes)):
    data[i]['symbol'] = codes[i]

In [4]:
df = pd.read_csv('../../FinRL/concat_data.csv')
df=df[['date', 'open', 'high', 'low', 'close', 'volume',
       'positive', 'neutral', 'negative', 'tic']]
df['date'] = [x[:10] for x in df['date']]
df = df[(df['date']>='2022-01-01') & (df['date']<'2023-09-30')]

In [5]:
data = pd.read_csv('../../min_data_adjust.csv')

In [6]:
stock_data = data[data['symbol']=='AAPL']

In [7]:
stock_data

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AAPL,2022-01-03 09:00:00+00:00,176.23,176.23,176.1800,176.1800,1118.0,65.0,176.210000
1,AAPL,2022-01-03 09:02:00+00:00,176.30,176.31,176.2800,176.2800,1218.0,26.0,176.300000
2,AAPL,2022-01-03 09:03:00+00:00,176.25,176.27,176.2500,176.2700,814.0,30.0,176.260000
3,AAPL,2022-01-03 09:04:00+00:00,176.20,176.20,176.1200,176.1200,3744.0,114.0,176.180000
4,AAPL,2022-01-03 09:05:00+00:00,176.17,176.17,176.1700,176.1700,464.0,33.0,176.150000
...,...,...,...,...,...,...,...,...,...
343433,AAPL,2023-09-29 23:53:00+00:00,171.30,171.30,171.3000,171.3000,209.0,8.0,171.305766
343434,AAPL,2023-09-29 23:54:00+00:00,171.30,171.30,171.3000,171.3000,810.0,16.0,171.305889
343435,AAPL,2023-09-29 23:57:00+00:00,171.32,171.32,171.3200,171.3200,439.0,20.0,171.330957
343436,AAPL,2023-09-29 23:58:00+00:00,171.30,171.30,171.2699,171.2699,532.0,11.0,171.282998


In [8]:
stock_df = df[df['tic']=='AAPL']

In [9]:
stock_df

,date,open,high,low,close,volume,positive,neutral,negative,tic
17,2022-01-03,177.830002,182.880005,177.710007,182.009995,104487900,-2.525743,3.722111,-3.922445,AAPL
21,2022-01-04,182.630005,182.940002,179.119995,179.699997,99310400,-2.752612,3.370780,-3.351379,AAPL
26,2022-01-05,179.610001,180.169998,174.639999,174.919998,94537600,-2.561095,3.561730,-3.588621,AAPL
35,2022-01-06,172.699997,175.300003,171.639999,172.000000,96904000,-2.294448,3.207229,-3.612424,AAPL
42,2022-01-07,172.889999,174.139999,171.029999,172.169998,86709100,-2.325235,3.084295,-3.352122,AAPL
...,...,...,...,...,...,...,...,...,...,...
3028,2023-09-25,174.199997,176.970001,174.149994,176.080002,46172700,-2.361765,3.181928,-3.037302,AAPL
3034,2023-09-26,174.820007,175.199997,171.660004,171.960007,64588900,-1.893191,2.688069,-3.369864,AAPL
3045,2023-09-27,172.619995,173.039993,169.050003,170.429993,66921800,-3.139558,3.359877,-2.654129,AAPL
3049,2023-09-28,169.339996,172.029999,167.619995,170.690002,56294400,-2.045589,2.791628,-3.063628,AAPL


In [10]:
stock_df['pctchange'] = (stock_df['close'] - stock_df['open'])/stock_df['open']

C:\Users\12364\AppData\Local\Temp\ipykernel_22072\1920598173.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_df['pctchange'] = (stock_df['close'] - stock_df['open'])/stock_df['open']


# Technical Indicators

In [11]:
from finta import TA

In [12]:
stock_df['SMA42'] = TA.SMA(stock_df, 42)
stock_df['SMA5'] = TA.SMA(stock_df, 5)
stock_df['SMA15'] = TA.SMA(stock_df, 15)
stock_df['AO'] = TA.AO(stock_df)
stock_df['OVB'] = TA.OBV(stock_df)
stock_df[['VW_MACD','MACD_SIGNAL']] = TA.VW_MACD(stock_df)
stock_df['RSI'] = TA.RSI(stock_df)
stock_df['CMO'] = TA.CMO(stock_df)

C:\Users\12364\AppData\Local\Temp\ipykernel_22072\1998282740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_df['SMA42'] = TA.SMA(stock_df, 42)
C:\Users\12364\AppData\Local\Temp\ipykernel_22072\1998282740.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_df['SMA5'] = TA.SMA(stock_df, 5)
C:\Users\12364\AppData\Local\Temp\ipykernel_22072\1998282740.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [13]:
stock_df = stock_df.dropna()

In [14]:
stock_df.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'positive', 'neutral',
       'negative', 'tic', 'pctchange', 'SMA42', 'SMA5', 'SMA15', 'AO', 'OVB',
       'VW_MACD', 'MACD_SIGNAL', 'RSI', 'CMO'],
      dtype='object')

In [15]:
stock_df_train = stock_df[stock_df['date']<='2023-03-31']
stock_df_test = stock_df[stock_df['date']>'2023-03-31']

# Replay Buffer

In [16]:
from collections import deque

test = deque(maxlen=5)
for i in range(10):
    test.append(i)
    print(test)

deque([0], maxlen=5)
deque([0, 1], maxlen=5)
deque([0, 1, 2], maxlen=5)
deque([0, 1, 2, 3], maxlen=5)
deque([0, 1, 2, 3, 4], maxlen=5)
deque([1, 2, 3, 4, 5], maxlen=5)
deque([2, 3, 4, 5, 6], maxlen=5)
deque([3, 4, 5, 6, 7], maxlen=5)
deque([4, 5, 6, 7, 8], maxlen=5)
deque([5, 6, 7, 8, 9], maxlen=5)


In [23]:
import random
from collections import deque
import torch
import torch.optim as optim
import numpy as np

class Agent:

    def __init__(self, state_size, action_size, bs, lr, tau, gamma, N, kappa, device, visual=False):
        '''
        When dealing with visual inputs, state_size should work as num_of_frame
        '''
        self.state_size = state_size
        self.action_size = action_size
        self.bs = bs
        self.lr = lr
        self.tau = tau
        self.gamma = gamma
        self.device = device
        self.N = N
        self.tau = torch.linspace(0, 1, N+1)
        self.tau = (self.tau[:-1] + self.tau[1:]) / 2
        self.tau_hat = self.tau.to(device).unsqueeze(0) #(1, N)
        self.tau = tau
        self.kappa = kappa

        self.Q_local = Q_Network(self.state_size, self.action_size, N).to(self.device)
        self.Q_target = Q_Network(self.state_size, self.action_size, N).to(self.device)
        self.soft_update(1)
        self.optimizer = optim.Adam(self.Q_local.parameters(), self.lr)
        self.memory = deque(maxlen=100000)

    def act(self, state, eps=0):
        if random.random() > eps:
            state = torch.tensor(state, dtype=torch.float32).to(self.device)
            with torch.no_grad():
                action_values = self.Q_local(state)
                action_values = action_values.mean(dim=-1, keepdim=False).view(-1)
            return np.argmax(action_values.cpu().numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self):
        experiences = random.sample(self.memory, self.bs)
        states = torch.from_numpy(np.vstack([e[0] for e in experiences])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences]).astype(np.uint8)).float().to(self.device)

        quantiles_local = self.Q_local(states) #(batch_size, action_size, N)
        quantiles_local = torch.gather(input=quantiles_local, dim=1, index=actions.unsqueeze(1).repeat(1, 1, self.N)) #(batch_size, 1, N)

        with torch.no_grad():
            quantiles_target = self.Q_target(next_states) #(batch_size, action_size, N)
            next_actions = torch.max(quantiles_target.sum(dim=2, keepdim=True), dim=1, keepdim=True)[1] #(batch_size, 1, 1)
            quantiles_target = torch.gather(input=quantiles_target, index=next_actions.repeat(1,1,self.N), dim=1) #(batch_size, 1, N)
            quantiles_target = rewards.unsqueeze(1).repeat(1, 1, self.N) + self.gamma * (1 - dones.unsqueeze(1).repeat(1, 1, self.N)) * quantiles_target #(batch_size, 1, N)
            quantiles_target = quantiles_target.permute(0, 2, 1) #(batch_size, N, 1)

        diff = quantiles_target - quantiles_local #(batch_size, N, N)
        tau = self.tau_hat.unsqueeze(0).repeat(diff.size(0), 1, 1) #(batch_size, 1, N)
        loss = (tau - (diff<0).float()).abs() * self.huber(diff) #(batch_size, N, N)
        loss = loss.mean(dim=2, keepdim=False).sum(dim=1, keepdim=False) #(batch_size,)
        loss = loss.mean()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.soft_update(self.tau)

    def huber(self, u):
        if self.kappa > 0:
            flag = (u.abs()<self.kappa).float()
            loss = 0.5*u.pow(2) * flag + self.kappa*(u.abs()-0.5*self.kappa) * (1-flag)
        else:
            loss = u.abs()
        return loss


    def soft_update(self, tau):
        for target_param, local_param in zip(self.Q_target.parameters(), self.Q_local.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

In [24]:
# indicators = ['open', 'high', 'low', 'close', 'volume', 'positive', 'neutral', 'negative','SMA42', 'SMA5', 'SMA15', 'AO', 'OVB','VW_MACD',
#        'MACD_SIGNAL', 'RSI', 'CMO']

indicators = ['pctchange', 'volume', 'positive', 'neutral', 'negative','SMA42', 'SMA5', 'SMA15', 'AO', 'OVB','VW_MACD',
       'MACD_SIGNAL', 'RSI', 'CMO']

In [25]:
class Stock_Env:
    def __init__(self, initial_asset, data, cost):
        self.asset = initial_asset
        self.cash = initial_asset
        self.stock = 0
        self.data = data
        self.time = data.iloc[time_period]['date']
        self.cost = cost
        self.history=[]
        self.total_cost = 0
        self.initial_asset = initial_asset
        self.rowid = time_period
        self.action_space = np.array(list(range(11)))
    
    def reset(self):
        self.asset = self.initial_asset
        self.cash = self.initial_asset
        self.stock = 0
        self.time = self.data.iloc[100]['date']
        self.history=[]
        self.total_cost = 0    
        self.rowid = time_period
        return self.data[:time_period][indicators].values
    
    def step(self, action):
        done = False
        states = self.data.iloc[self.rowid]        
        self.rowid +=1
        if self.rowid == len(self.data)-1:
            done = True
        next_state = self.data.iloc[self.rowid]
        last_asset = self.asset
        price = next_state['open']
        old_asset = self.cash + self.stock*price
        self.asset = old_asset
        target_value = action*0.1*self.asset
        distance = target_value - self.stock*price
        stock_distance = int(distance/(price*(1+self.cost)))
        self.stock += stock_distance
        self.cash = self.cash - distance - np.abs(stock_distance*self.cost*price)
        self.asset = self.cash+self.stock*price
        market_value = self.stock * next_state['close']
        self.asset = market_value + self.cash
        reward = (self.asset - last_asset)/last_asset
        self.time = next_state['date']
        # self.stock = stock
        return (self.data[self.rowid-time_period:self.rowid][indicators].values, reward, done)

In [26]:
#env = gym.make()
env = Stock_Env(1000000, stock_df_train, 0.002)
env_test = Stock_Env(1000000, stock_df_test, 0.002)
num_episode = 5
max_t = 1000
reward_log = []

for _ in range(num_episode):
    
    # initialize
    env.reset()
    t = 0
    episodic_reward = 0
    
    for t in range(max_t):
        
        #env.render()
        action = np.random.randint(11) # random action
        _, reward, done = env.step(action)
        episodic_reward += reward
        if done:
            break
    
    reward_log.append(episodic_reward)

In [45]:
agent = Agent(1, len(env.action_space), 64, 0.001, 0.001, 0.99, 51, 1, 'cuda', True)

In [46]:
import warnings
warnings.filterwarnings('ignore')
#env = gym.make()
num_episode = 20000
max_t = 1000
reward_log = []
average_log = [] # monitor training process
eps = 1
eps_decay = 0.995
eps_min = 0.01
C = 4 # update weights every C steps

def validation(env, agent):
    rewards_log = []
    average_log = []
    episodic_reward = 0
    done = False
    t = 0
    state = env.reset()
    while not done and t < max_t:
        t += 1
        action = agent.act(state, eps)
        frame, reward, done = env.step(action)
        next_state = frame
        state = next_state.copy()
        episodic_reward += reward
    return env.asset


def train(env, agent, num_episode, eps_init, eps_decay, eps_min, max_t, num_frame=1, constant=0):
    rewards_log = []
    average_log = []
    eps = eps_init

    for i in range(1, 1 + num_episode):

        episodic_reward = 0
        done = False
        frame = env.reset()
        state_deque = deque(maxlen=num_frame)
        for _ in range(num_frame):
            state_deque.append(frame)
        state = np.stack(state_deque, axis=0)
        state = np.expand_dims(state, axis=0)
        t = 0

        while not done and t < max_t:

            t += 1
            action = agent.act(state, eps)
            frame, reward, done = env.step(action)
            state_deque.append(frame)
            next_state = np.stack(state_deque, axis=0)
            next_state = np.expand_dims(next_state, axis=0)
            agent.memory.append((state, action, reward, next_state, done))

            if t % 5 == 0 and len(agent.memory) >= agent.bs:
                agent.learn()
                agent.soft_update(agent.tau)

            state = next_state.copy()
            episodic_reward += reward
        
        val_asset = validation(env_test, agent)

        rewards_log.append(episodic_reward)
        average_log.append(np.mean(rewards_log[-100:]))
        print('\rEpisode {}, Reward {:.3f}, Average Reward {:.3f}, Asset {:.2f}, Validation Asset {:.2f}'.format(i, episodic_reward, average_log[-1], env.asset, val_asset), end='')
        if i % 100 == 0:
            print()

        eps = max(eps * eps_decay, eps_min)

    return rewards_log

In [47]:
train(env, agent, num_episode, eps, eps_decay, eps_min, max_t, num_frame=1, constant=C)

Episode 100, Reward -0.209, Average Reward -0.198, Asset 796622.07, Validation Asset 923438.31
Episode 200, Reward -0.205, Average Reward -0.154, Asset 801698.41, Validation Asset 892236.310
Episode 300, Reward -0.268, Average Reward -0.116, Asset 746634.72, Validation Asset 975563.627
Episode 400, Reward -0.042, Average Reward -0.085, Asset 939428.02, Validation Asset 939991.534
Episode 500, Reward -0.091, Average Reward -0.104, Asset 890522.68, Validation Asset 939189.915
Episode 600, Reward -0.065, Average Reward -0.086, Asset 924254.38, Validation Asset 905469.983
Episode 700, Reward 0.105, Average Reward -0.071, Asset 1097075.53, Validation Asset 977096.363
Episode 800, Reward 0.014, Average Reward -0.085, Asset 996283.92, Validation Asset 1020814.374
Episode 900, Reward 0.008, Average Reward -0.086, Asset 984810.98, Validation Asset 924618.6906
Episode 1000, Reward -0.147, Average Reward -0.102, Asset 850923.08, Validation Asset 1026252.38
Episode 1100, Reward -0.073, Average Rew

KeyboardInterrupt: 

In [ ]:
# eps_init = eps
# constant = C
# num_frame =1

# rewards_log = []
# average_log = []
# eps = eps_init

# for i in range(1, 1 + num_episode):
#     episodic_reward = 0
#     done = False
#     frame = env.reset()
#     state_deque = deque(maxlen=num_frame)
#     for _ in range(num_frame):
#         state_deque.append(frame)
#     state = np.stack(state_deque, axis=0)
#     state = np.expand_dims(state, axis=0)
#     t = 0

#     while not done and t < max_t:

#         t += 1
#         action = agent.act(state, eps)
#         frame, reward, done = env.step(action)
#         state_deque.append(frame)
#         next_state = np.stack(state_deque, axis=0)
#         next_state = np.expand_dims(next_state, axis=0)
#         agent.memory.append((state, action, reward, next_state, done))

#         if t % 5 == 0 and len(agent.memory) >= agent.bs:
#             agent.learn()
#             agent.soft_update(agent.tau)

#         state = next_state.copy()
#         episodic_reward += reward

#     rewards_log.append(episodic_reward)
#     average_log.append(np.mean(rewards_log[-100:]))
#     print('\rEpisode {}, Reward {:.3f}, Average Reward {:.3f}'.format(i, episodic_reward, average_log[-1]), end='')
#     if i % 100 == 0:
#         print()

#     eps = max(eps * eps_decay, eps_min)